In [2]:
import pandas as pd

In [3]:
data_new = pd.read_csv('../../data/scrapped/voitures_clean.csv')

In [4]:
import joblib
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# =======================
# CHARGER LES DONNÉES
# =======================
df = data_new  # ton dataframe

df = df.drop(columns=['Ville', 'Année-Modèle'])
df['logPrix'] = np.log1p(df['Prix'])

X = df.drop(columns=['Prix', 'logPrix'])
y = df['logPrix']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# =======================
# TARGET ENCODING
# =======================
global_mean = y_train.mean()

train_temp = X_train.copy()
train_temp['logPrix'] = y_train

marque_te = train_temp.groupby('Marque')['logPrix'].mean().to_dict()
modele_te = train_temp.groupby('Modèle')['logPrix'].mean().to_dict()

X_train['Marque_te'] = X_train['Marque'].map(marque_te)
X_test['Marque_te'] = X_test['Marque'].map(marque_te).fillna(global_mean)

X_train['Modèle_te'] = X_train['Modèle'].map(modele_te)
X_test['Modèle_te'] = X_test['Modèle'].map(modele_te).fillna(global_mean)

X_train = X_train.drop(columns=['Marque', 'Modèle'])
X_test = X_test.drop(columns=['Marque', 'Modèle'])

# =======================
# ONE HOT ENCODING
# =======================
cat_cols = [
    'Boite de vitesses',
    'Type de carburant',
    'Origine',
    'Première main',
    'Ville_grouped'
]

X_train = pd.get_dummies(X_train, columns=cat_cols, drop_first=True)
X_test = pd.get_dummies(X_test, columns=cat_cols, drop_first=True)

X_train, X_test = X_train.align(X_test, join='left', axis=1)
X_test = X_test.fillna(0)

# =======================
# SCALING
# =======================
num_cols = [
    'Kilométrage',
    'Age_vehicule',
    'Nombre de portes',
    'Puissance fiscale',
    'Marque_te',
    'Modèle_te'
]

scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

# =======================
# MODEL
# =======================
model = RandomForestRegressor(
    n_estimators=50,
    max_depth=20,
    max_features=0.33,
    random_state=0,
    n_jobs=-1
)

model.fit(X_train, y_train)

# =======================
# SAVE FILES
# =======================
joblib.dump(model, "model_rf.pkl")
joblib.dump(scaler, "scaler.pkl")
joblib.dump(X_train.columns.tolist(), "features.pkl")
joblib.dump(marque_te, "marque_te.pkl")
joblib.dump(modele_te, "modele_te.pkl")
joblib.dump(global_mean, "global_mean.pkl")

print("✅ Modèle sauvegardé avec succès")


✅ Modèle sauvegardé avec succès
